In [2]:
from google.colab import drive
drive.mount("ImageColoring")

Mounted at ImageColoring


In [3]:
"Image colorization using Autoencoders. Transfer learning using VGG."
"Importing Necessary Libraries"
from keras.layers import Conv2D, UpSampling2D, Input
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
import numpy as np
import tensorflow as tf
import keras
import os

In [10]:
"Replacing the encoder part with Feature Extraxtor of VGG"
from keras.applications.vgg16 import VGG16
vggmodel = VGG16()
newmodel = Sequential() 
#num = 0
for i, layer in enumerate(vggmodel.layers):
    if i<19:          #Only up to 19th layer to include feature extraction only
      newmodel.add(layer)
newmodel.summary()
for layer in newmodel.layers:
  layer.trainable=False  #We don't want to train these layers again, so False. 

553476096/553467096 [==============================] - 6s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_______________________________________________________

In [4]:
"""
VGG16 is expecting an image of 3 dimension with size 224x224 as an input, 
in preprocessing we have to scale all images to 224 instead of 256
"""
ROOT_DIR = 'ImageColoring/MyDrive/ImageColoring/'
#Normalize images - divide by 255
train_datagen = ImageDataGenerator(rescale=1. / 255)

train = train_datagen.flow_from_directory(ROOT_DIR, target_size=(224, 224), batch_size=1000, class_mode=None)

Found 7144 images belonging to 2 classes.


In [7]:
"Convert from RGB to Lab: LAB image is a grey image in L channel and all color info stored in A and B channels"
X =[]
Y =[]
for img in train[0]:
  try:
      lab = rgb2lab(img)
      X.append(lab[:,:,0]) 
      Y.append(lab[:,:,1:] / 128) #A and B values range from -127 to 128, 
      #so we divide the values by 128 to restrict values to between -1 and 1.
  except:
     print('error')
X = np.array(X)
Y = np.array(Y)
X = X.reshape(X.shape+(1,)) #dimensions to be the same for X and Y
print(X.shape)
print(Y.shape)

(1000, 224, 224, 1)
(1000, 224, 224, 2)


In [8]:
#now we have one channel of L in each layer but, VGG16 is expecting 3 dimension, 
#so we repeated the L channel two times to get 3 dimensions of the same L channel
vggfeatures = []
for i, sample in enumerate(X):
  sample = gray2rgb(sample)
  sample = sample.reshape((1,224,224,3))
  prediction = newmodel.predict(sample)
  prediction = prediction.reshape((7,7,512))
  vggfeatures.append(prediction)
vggfeatures = np.array(vggfeatures)
print(vggfeatures.shape)

(1000, 7, 7, 512)


In [9]:
#Decoder
model = Sequential()

model.add(Conv2D(256, (3,3), activation='relu', padding='same', input_shape=(7,7,512)))
model.add(Conv2D(128, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(16, (3,3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 7, 7, 256)         1179904   
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 128)         295040    
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        73792     
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 28, 28, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 32)        18464     
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 56, 56, 32)       

In [10]:
model.compile(optimizer='Adam', loss='mse' , metrics=['accuracy'])
model.fit(vggfeatures, Y, verbose=1, epochs=2000, batch_size=16)


Epoch 1/2000
63/63 [==============================] - 3s 16ms/step - loss: 0.0455 - accuracy: 0.6002
Epoch 2/2000
63/63 [==============================] - 1s 12ms/step - loss: 0.0104 - accuracy: 0.6775
Epoch 3/2000
63/63 [==============================] - 1s 12ms/step - loss: 0.0094 - accuracy: 0.6925
Epoch 4/2000
63/63 [==============================] - 1s 12ms/step - loss: 0.0087 - accuracy: 0.7045
Epoch 5/2000
63/63 [==============================] - 1s 12ms/step - loss: 0.0082 - accuracy: 0.7088
Epoch 6/2000
63/63 [==============================] - 1s 12ms/step - loss: 0.0077 - accuracy: 0.7181
Epoch 7/2000
63/63 [==============================] - 1s 12ms/step - loss: 0.0074 - accuracy: 0.7279
Epoch 8/2000
63/63 [==============================] - 1s 12ms/step - loss: 0.0068 - accuracy: 0.7375
Epoch 9/2000
63/63 [==============================] - 1s 12ms/step - loss: 0.0065 - accuracy: 0.7460
Epoch 10/2000
63/63 [==============================] - 1s 13ms/step - loss: 0.0062 - accura

In [12]:
model.save(ROOT_DIR+'colorize_autoencoder_2000.model')

INFO:tensorflow:Assets written to: ImageColoring/MyDrive/ImageColoring/colorize_autoencoder_2000.model/assets


In [5]:
ROOT_DIR = 'ImageColoring/MyDrive/ImageColoring/'

In [6]:
#Predicting using saved model.
model = tf.keras.models.load_model(ROOT_DIR+'colorize_autoencoder_2000.model',
                                   custom_objects=None,
                                   compile=True)


In [8]:
testpath = 'ImageColoring/MyDrive/ImageColoring/gray/'
files = os.listdir(testpath)
files

['109.jpg', '107.jpg', '106.jpg', '110.jpg', '108.jpg']

In [11]:
for idx, file in enumerate(files):
    test = img_to_array(load_img(testpath+file))
    test = resize(test, (224,224), anti_aliasing=True)
    test*= 1.0/255
    lab = rgb2lab(test)
    l = lab[:,:,0]
    L = gray2rgb(l)
    L = L.reshape((1,224,224,3))
    #print(L.shape)
    vggpred = newmodel.predict(L)
    ab = model.predict(vggpred)
    #print(ab.shape)
    ab = ab*128
    cur = np.zeros((224, 224, 3))
    cur[:,:,0] = l
    cur[:,:,1:] = ab
    imsave('ImageColoring/MyDrive/ImageColoring/gray/'+str(idx)+".jpg", lab2rgb(cur))

/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 2 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
/usr/local/lib/python3.7/dist-packages/skimage/color/colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 1 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))


In [ ]:
!pip freeze > ImageColoring/MyDrive/ImageColoring/requirements.txt